## 데이터 탐색
#### 이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.
#### 추천 모델을 만들기 전에 데이터의 기본적인 정보

#####  - 유저수, 아티스트수, 인기 많은 아티스트
#####  - 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
#####  - 유저 play 횟수 중앙값에 대한 통계

In [1]:
import os
import pandas as pd
import numpy as np

## 1) 데이터 준비와 전처리

In [2]:
# 평점 데이터 읽어오기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
# 영화 데이터 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
# rating이 3점 이상만 데이터만 남기기
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# rating 컬럼의 이름을 count로 바꾸기
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

## 2) 분석해 봅시다.

#### - ratings에 있는 유니크한 영화 개수
#### - ratings에 있는 유니크한 사용자 수
#### - 가장 인기 있는 영화 30개(인기순)

In [6]:
# ratings에 있는 유니크한 영화 개수
ratings["movie_id"].nunique()

3628

In [7]:
# rating에 있는 유니크한 사용자 수
ratings["user_id"].nunique()

6039

In [8]:
# 가장 인기 있는 영화 30개(인기순)

# rating과 movies를 합친다.
ratings_by_count = pd.merge(ratings, movies, how="left", on="movie_id")

# rating을 count를 기준으로 내림차순해준다.
ratings_by_count.sort_values(by="count", ascending=False, inplace=True)
ratings_by_count.reset_index(inplace=True, drop=True)

# 가장 인기 있는 상위 30개
ratings_by_count.head(30)

,user_id,movie_id,count,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2339,1952,5,974420466,Midnight Cowboy (1969),Drama
2,4671,1210,5,963794984,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
3,2340,715,5,974421414,"Horseman on the Roof, The (Hussard sur le toit...",Drama
4,2340,2987,5,974422967,Who Framed Roger Rabbit? (1988),Adventure|Animation|Film-Noir
5,4671,380,5,963795314,True Lies (1994),Action|Adventure|Comedy|Romance
6,1293,524,5,974788401,Rudy (1993),Drama
7,1293,527,5,974788915,Schindler's List (1993),Drama|War
8,2339,3929,5,974421058,"Bank Dick, The (1940)",Comedy
9,2339,3922,5,974421095,Bikini Beach (1964),Comedy


In [9]:
# 사용하는 컬럼만 남긴다.
using_cols = ['user_id', 'title', 'count']
data = ratings_by_count[using_cols]
data.head(10)

,user_id,title,count
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2339,Midnight Cowboy (1969),5
2,4671,Star Wars: Episode VI - Return of the Jedi (1983),5
3,2340,"Horseman on the Roof, The (Hussard sur le toit...",5
4,2340,Who Framed Roger Rabbit? (1988),5
5,4671,True Lies (1994),5
6,1293,Rudy (1993),5
7,1293,Schindler's List (1993),5
8,2339,"Bank Dick, The (1940)",5
9,2339,Bikini Beach (1964),5


## 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [11]:
## user_id의 끝 숫자가 6040이니 6041부터 추가해준다.
data["user_id"].max()

6040

In [12]:
my_favorite = ['game of throne', 'inception', 'tenet', 'intersteller', 'joe black']

my_playlist = pd.DataFrame({\
                            'user_id': ['6041']*5, \
                            'title': my_favorite, \
                            'count':[4]*5
                           })

if not data.isin({'user_id':['6041']})['user_id'].any():  # user_id에 '6041'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,title,count
836473,4194,James and the Giant Peach (1996),3
836474,1727,Never Been Kissed (1999),3
836475,1727,Jury Duty (1995),3
836476,4194,"Opposite of Sex, The (1998)",3
836477,3080,Keeping the Faith (2000),3
0,6041,game of throne,4
1,6041,inception,4
2,6041,tenet,4
3,6041,intersteller,4
4,6041,joe black,4


## 4) CSR matrix를 직접 만들어 봅시다.

In [13]:
# CSR matrix를 넣기 위한 scipy 패키지
from scipy.sparse import csr_matrix

In [14]:
# 고유한 유저, movie 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

# user_id, movie title indexing 하는 코드. (idx는 index의 약자)
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# 인덱싱이 잘 되었는지 확인하기. 
print(user_to_idx[2339]) 
print(movie_to_idx['Schindler\'s List (1993)'])

1
7


In [15]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해주기. 
temp_artist_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('movie title column indexing OK!!')
    data['title'] = temp_artist_data
else:
    print('movie title column indexing Fail!!')

data

user_id column indexing OK!!
movie title column indexing OK!!


,user_id,title,count
0,0,0,5
1,1,1,5
2,2,2,5
3,3,3,5
4,3,4,5
...,...,...,...
0,6039,3628,4
1,6039,3629,4
2,6039,3630,4
3,6039,3631,4


In [16]:
num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

## csr matrix로 만들기
csr_data = csr_matrix((data["count"], (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [18]:

from implicit.als import AlternatingLeastSquares
import os

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [19]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [20]:
# 비슷한 영화 리스트를 알아보기
favorite_movie = 'intersteller'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3631, 1.0),
 (3629, 0.9910142),
 (3630, 0.98896486),
 (3628, 0.98867184),
 (3632, 0.98802793),
 (3621, 0.9649856),
 (3624, 0.96472514),
 (3625, 0.96452916),
 (3622, 0.96357983),
 (3404, 0.9632902),
 (3620, 0.9629797),
 (3420, 0.96273184),
 (3416, 0.9619906),
 (3417, 0.9603818),
 (3411, 0.9569529)]

In [21]:
# artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성한다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['intersteller',
 'inception',
 'tenet',
 'game of throne',
 'joe black',
 'Century (1993)',
 'Number Seventeen (1932)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 "Brother's Kiss, A (1997)",
 'Male and Female (1919)',
 'War at Home, The (1996)',
 'Neon Bible, The (1995)',
 "Another Man's Poison (1952)",
 'Daens (1992)',
 'Second Best (1994)']

In [22]:
# 위의 2cell을 함수로 정리해서 만든다.
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [23]:
get_similar_movie('intersteller')

['intersteller',
 'inception',
 'tenet',
 'game of throne',
 'joe black',
 'Century (1993)',
 'Number Seventeen (1932)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 "Brother's Kiss, A (1997)",
 'Male and Female (1919)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [24]:
user = user_to_idx['6041']
# recommend에서는 user*item CSR Matrix를 받는다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[idx_to_movie[i[0]] for i in movie_recommended]

['L.A. Confidential (1997)',
 'Anna and the King (1999)',
 'Fish Called Wanda, A (1988)',
 'Xiu Xiu: The Sent-Down Girl (Tian yu) (1998)',
 'Stuart Little (1999)',
 'Thieves (Voleurs, Les) (1996)',
 'Groundhog Day (1993)',
 'Beautiful Thing (1996)',
 'Great White Hype, The (1996)',
 'Man on the Moon (1999)',
 'Jakob the Liar (1999)',
 'Walk on the Moon, A (1999)',
 'Indochine (1992)',
 'Phantom of the Opera, The (1943)',
 'Road Trip (2000)',
 'Shakespeare in Love (1998)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'Black Cauldron, The (1985)',
 'Rain Man (1988)',
 'Last Night (1998)']

In [25]:
# explain 메소드를 사용하면 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.
tenet = movie_to_idx['tenet']
explain = als_model.explain(user, csr_data, itemid=tenet)
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('inception', 4.704286493311009e-05),
 ('joe black', 3.739641557804642e-05),
 ('tenet', 3.322787654374501e-05),
 ('intersteller', 1.7052091410344204e-05),
 ('game of throne', 1.2410181426321244e-05)]